In [1]:
##################################
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd 
import numpy as np
import random
##################################

In [2]:
df = pd.read_csv("bank.csv")
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [3]:
job = [ "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown"]

for ii in range(len(job)): 
    df.job[df.job == job[ii]] = ii

marital = ["divorced","married","single","unknown"]

for ii in range(len(marital)):
    df.marital[df.marital == marital[ii]] = ii

education =  ["primary","secondary","tertiary","unknown"]

for ii in range(len(education)):
    df.education[df.education == education[ii]] = ii

df.default[df.default == "no" ] = 0
df.default[df.default == "yes" ] = 1

df.housing[df.housing == "no" ] = 0
df.housing[df.housing == "yes" ] = 1

df
#sampled = df.sample(1000)
#sampled.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,7,1,1,0,4789,1,yes,cellular,11,may,220,1,339,4,failure,no
2,35,4,2,2,0,1350,1,no,cellular,16,apr,185,1,330,1,failure,no
3,30,4,1,2,0,1476,1,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,1,1,1,0,0,1,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,7,1,1,0,-333,1,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,6,1,2,1,-3313,1,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,9,1,1,0,295,0,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,1,1,1,0,1137,0,no,cellular,6,feb,129,4,211,3,other,no


In [4]:
#new_df = np.array(sampled[['age','balance']])
#new_df

In [5]:
#distortions = []
#for i in range(1, 11):
#    km = KMeans(n_clusters=i)
#    km.fit(new_df)
#    distortions.append(km.inertia_)
#plt.plot(range(1, 11), distortions, 'bx-')
#plt.xlabel('Number of clusters')
#plt.ylabel('Distortion')
#plt.tight_layout()

#plt.show()